In [ ]:
from IPython.display import display, Markdown, clear_output
import utils
import ipywidgets as ipw
import widgets

# Sample preparation

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
ACTIONS = {
    "SPUTTERING": "Sputtering",
    "ANNEALING": "Annealing",
    "DEPOSITION": "Deposition",
    "DOSING": "Dosing"
}

In [ ]:
class ActionsAccordion(ipw.Accordion):
    def __init__(self):
        super().__init__()
    def load_accordion(self, openbis_session, process_actions, possible_actions):
        accordion_widgets = []
        for process_action_identifier in process_actions:
            process_action_data = utils.get_openbis_object_data(openbis_session, process_action_identifier)
            process_action_title = process_action_data["props"]["$name"] + " (" + process_action_data["registration_date"] + ")"
            self.set_title(len(accordion_widgets), process_action_title)
            object_type = possible_actions[process_action_data["type"]]
            properties_widgets = widgets.ObjectPropertiesWidgets(object_type, disabled = True)
            properties_widgets.get_properties_widgets()
            
            utils.load_widget_values(properties_widgets.properties_widgets_detailed_dict, process_action_data["props"])
            
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = True)
            component_selector.load_dropdown_box()
            
            for parent_identifier in process_action_data["parents"]:
                parent_data = utils.get_openbis_object_data(openbis_session, parent_identifier)
                if parent_data["type"] == "COMPONENT":
                    component_selector.dropdown.value = parent_data["permId"]
            
            action_widgets = ipw.VBox([component_selector, properties_widgets])
            accordion_widgets.append(action_widgets)
        self.children = accordion_widgets

class ObservablesAccordion(ipw.Accordion):
    def __init__(self):
        super().__init__()
    def load_accordion(self, openbis_session, process_observables):
        accordion_widgets = []
        
        for process_observable_identifier in process_observables:
            process_observable_data = utils.get_openbis_object_data(openbis_session, process_observable_identifier)
            process_observable_title = process_observable_data["props"]["$name"] + " (" + process_observable_data["registration_date"] + ")"
            self.set_title(len(accordion_widgets), process_observable_title)
            properties_widgets = widgets.ObjectPropertiesWidgets("Observable", disabled = True)
            properties_widgets.get_properties_widgets()
            
            utils.load_widget_values(properties_widgets.properties_widgets_detailed_dict, process_observable_data["props"])
            
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = True)
            component_selector.load_dropdown_box()
            
            for parent_identifier in process_observable_data["parents"]:
                parent_data = utils.get_openbis_object_data(openbis_session, parent_identifier)
                if parent_data["type"] == "COMPONENT":
                    component_selector.dropdown.value = parent_data["permId"]
            
            observable_widgets = ipw.VBox([component_selector, properties_widgets])
            accordion_widgets.append(observable_widgets)
        self.children = accordion_widgets

class ProcessStepWidgets(ipw.VBox):
    def __init__(self, process_data, actions, observables):
        super().__init__()
        self.actions = actions
        self.observables = observables
        self.protocol_identifier = None
        self.actions_label = ipw.HTML("<b><span style='font-size:14px;'>Actions</span></b>")
        self.observables_label = ipw.HTML("<b><span style='font-size:14px;'>Observables</span></b>")
        self.process_data = process_data
        self.title = self.process_data["props"]["$name"] + " (" + self.process_data["registration_date"] + ")"
        self.children = [self.actions_label, self.actions, self.observables_label, self.observables]
    
    def find_protocol(self, openbis_session):
        for process_parent_identifier in self.process_data["parents"]:
            process_parent_data = utils.get_openbis_object_data(openbis_session, process_parent_identifier)
            if process_parent_data["type"] == "SAMPLE_PROCESS":
                self.protocol_identifier = process_parent_data["permId"]
                break

class SamplePreparationHistoryWidgets(ipw.Accordion):
    def __init__(self):
        super().__init__()
        
    def create_process_step_widgets(self, openbis_session, sample_processes):
        accordion_items = []
        protocol_process_items = {}
        for process_identifier, sample_process in sample_processes.items():
            process_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_identifier)
            
            # Actions
            process_actions_accordion = ActionsAccordion()
            process_actions_accordion.load_accordion(OPENBIS_SESSION, sample_process["actions"], ACTIONS)
            
            # Observables
            process_observables_accordion = ObservablesAccordion()
            process_observables_accordion.load_accordion(OPENBIS_SESSION, sample_process["observables"])
            
            # Process Step Widgets
            process_step_widgets = ProcessStepWidgets(process_data, process_actions_accordion, process_observables_accordion)
            process_step_widgets.find_protocol(OPENBIS_SESSION)
            
            if process_step_widgets.protocol_identifier:
                process_step_details = {"name": process_data["props"]["$name"], "registration_date": process_data["registration_date"], "items": process_step_widgets}
                protocol_process_items.setdefault(process_step_widgets.protocol_identifier, []).append(process_step_details)
            else:
                self.set_title(len(accordion_items), process_step_widgets.title)
                accordion_items.append(process_step_widgets)
        
        for protocol_process_identifier, protocol_process_steps_details in protocol_process_items.items():
            protocol_process_data = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_process_identifier)
            protocol_process_accordion = ipw.Accordion()
            protocol_process_accordion_list = []
            for protocol_process_step_details in protocol_process_steps_details:
                protocol_process_title = protocol_process_step_details["name"] + " (" + protocol_process_step_details["registration_date"] + ")"
                protocol_process_accordion.set_title(len(protocol_process_accordion_list), protocol_process_title)
                protocol_process_accordion_list.append(protocol_process_step_details["items"])
            protocol_process_accordion.children = protocol_process_accordion_list
            
            process_step_title = protocol_process_data["props"]["$name"] + " (" + protocol_process_data["registration_date"] + ")"
            self.set_title(len(accordion_items), process_step_title)
            accordion_items.append(protocol_process_accordion)
        self.children = accordion_items

def get_sample_processes(sample_identifier):
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid)
        
        if current_object_data["type"] == "SAMPLE_PROCESS" and current_object_data["permId"] not in sample_processes:
            process_actions = []
            process_observables = []
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier)
                if child_object_data["type"] in ACTIONS.keys():
                    process_actions.append(child_object_data["permId"])
                elif child_object_data["type"] == "OBSERVABLE":
                    process_observables.append(child_object_data["permId"])
            
            if process_actions or process_observables:   
                sample_processes[current_object_data["permId"]] = {
                    "actions": process_actions,
                    "observables": process_observables
                }
        
        for parent_identifier in current_object.parents:
            parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

In [ ]:
# sample_processes = get_sample_processes("20250303102131254-33252")

In [ ]:
sample_selector = widgets.ObjectSelectionWidget("Sample")
sample_selector.load_dropdown_box()

sample_preparation_history = SamplePreparationHistoryWidgets()

add_process_button = utils.Button(
    description = 'Add protocol process', disabled = False, button_style = '', 
    tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
)

add_process_step_button = utils.Button(
    description = 'Add process step', disabled = False, button_style = '', 
    tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
)

add_processes_buttons = ipw.HBox([add_process_button, add_process_step_button])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
new_sample_processes = ipw.Accordion()

In [ ]:
def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        sample_preparation_history.children = []
    else:
        sample_processes = get_sample_processes(sample_selector.dropdown.value)
        sample_preparation_history.create_process_step_widgets(OPENBIS_SESSION, sample_processes)
        sample_preparation_history.selected_index = None

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
def add_process_step(b):
    add_action = ipw.Button(description = "Add action")
    remove_action = ipw.Button(description = "Remove action")
    action_buttons = ipw.HBox([add_action, remove_action])
    remove_process_step_button = ipw.Button(description = "Remove process step")
    
    current_accordion_widgets = list(new_sample_processes.children)
    number_process_steps = len(current_accordion_widgets)
    
    def remove_process_step(b):
        print(number_process_steps)
        current_accordion_widgets = list(new_sample_processes.children)
        del current_accordion_widgets[number_process_steps]
        new_sample_processes.children = current_accordion_widgets
    
    remove_process_step_button.on_click(remove_process_step)
    process_step_widgets = ipw.VBox([action_buttons, remove_process_step_button])
    
    new_sample_processes.set_title(number_process_steps, number_process_steps)
    current_accordion_widgets.append(process_step_widgets)
    new_sample_processes.children = current_accordion_widgets

In [ ]:
display(Markdown("## Select sample"))
sample_selector.dropdown.observe(load_sample_metadata, names = 'value')
display(sample_selector)

display(Markdown("## Sample history"))
display(sample_preparation_history)

display(Markdown("## Register new sample processing steps"))
add_process_step_button.on_click(add_process_step)
display(new_sample_processes)
display(add_processes_buttons)
quit_button.on_click(close_notebook)
display(save_close_buttons_hbox)
display(increase_buttons_size)